In [25]:
import pandas as pd
import numpy as np
import psycopg2
from datetime import datetime, timedelta

def connect_prd_order():
    try:
        connection = psycopg2.connect(
            host="35.187.250.81",
            database="prd_order",
            user="akhiyar_waladi",
            password="nd4n6fk9")


        print("You are connected")

    except (Exception, psycopg2.Error) as error :
        print("Error while connecting to PostgreSQL", error)
        
    return connection

In [26]:
main = """
    select 
        tbto_voucher_code, tbto_ponta_id, tbto_no, tbto_voucher_usage, tbto_create_date
    from 
        tb_transaction_order tto 
    where 
        tbto_create_date between '{}' and '{}'
        and tbto_status not in ('18','10','11')
        and tbto_voucher_code not in ('')
""".format('2021-01-01', '2021-06-17')
cpo = connect_prd_order()
df_window = pd.read_sql_query(main,cpo)

You are connected


In [18]:
df_window_g1 = df_window.groupby('tbto_voucher_code')\
                .agg({'tbto_no':'count', 'tbto_voucher_usage':'sum', 'tbto_voucher_usage':'sum'})\
                .reset_index()

In [19]:
df_window_g2 = df_window.groupby('tbto_voucher_code')\
                .agg({'tbto_no':'count', 'tbto_voucher_usage':'sum', 'tbto_voucher_usage':'min'})\
                .reset_index()

### sum usage voucher count > 1

In [22]:
df_window_g1[df_window_g1['tbto_no'] > 1]

,tbto_voucher_code,tbto_no,tbto_voucher_usage
47,B0000154DLIX,2,28000.0
67,B0000B6CCOE4,2,6000.0
367,B00098740W5Y,2,2000.0
457,B000C6BE6A86,2,1000.0
830,B0017A833X3S,2,16000.0
...,...,...,...
359527,BFFBB2D4A2HC,2,2000.0
360358,BFFCC618E17D,2,1000.0
361485,BFFE3BC67QBM,2,26000.0
362009,BFFEF006DMI7,2,22000.0


### nilai satuan voucher > 1

In [23]:
df_window_g2[df_window_g2['tbto_no'] > 1]

,tbto_voucher_code,tbto_no,tbto_voucher_usage
47,B0000154DLIX,2,14000.0
67,B0000B6CCOE4,2,3000.0
367,B00098740W5Y,2,1000.0
457,B000C6BE6A86,2,500.0
830,B0017A833X3S,2,8000.0
...,...,...,...
359527,BFFBB2D4A2HC,2,1000.0
360358,BFFCC618E17D,2,500.0
361485,BFFE3BC67QBM,2,13000.0
362009,BFFEF006DMI7,2,11000.0


### hitung loss dari sum usage semua voucher count > 1 - nilai satuan usage voucher count > 1

In [24]:
loss_series = pd.Series(\
(df_window_g1[df_window_g1['tbto_no'] > 1]['tbto_voucher_usage'].sum()) - \
(df_window_g2[df_window_g2['tbto_no'] > 1]['tbto_voucher_usage'].sum()))\
        .astype(int).apply(lambda x : "{:,d}".format(x))

pd.DataFrame({'loss_voucher':loss_series})

,loss_voucher
0,"10,415,100"
